In [1]:
import numpy as np
import jax.numpy as jnp
import pytest
import ticktack
from ticktack import fitting

In [2]:
cbm = ticktack.load_presaved_model('Guttler14', production_rate_units='atoms/cm^2/s')
sf = fitting.SingleFitter(cbm, 'Guttler14', hemisphere="north")

INFO[2022-02-01 20:31:51,505]: Unable to initialize backend 'tpu_driver': Not found: Unable to find driver in registry given worker: 
INFO[2022-02-01 20:31:51,506]: Unable to initialize backend 'gpu': Not found: Could not find registered platform with name: "cuda". Available platform names are: Host Interpreter
INFO[2022-02-01 20:31:51,507]: Unable to initialize backend 'tpu': Invalid argument: TpuPlatform is not available.
WARNING[2022-02-01 20:31:51,508]: No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [3]:
sf.load_data("data/775AD-late-N/Guttler15_Kauri.csv")

In [5]:
sf.time_data.shape

(20,)

In [2]:
cbm = ticktack.load_presaved_model('Guttler14', production_rate_units='atoms/cm^2/s')
sf = fitting.SingleFitter(cbm, 'Guttler14', hemisphere="north")
sf.time_data = jnp.arange(200, 210)
sf.d14c_data_error = jnp.ones((sf.time_data.size,))
sf.d14c_data = jnp.array([-169.81482498, -168.05109886, -163.81278239, -158.13313339,
                          -153.23525037, -150.89762995, -151.11804461, -152.1036496 ,
                          -151.60492619, -151.60492619])
sf.start = np.nanmin(sf.time_data)
sf.end = np.nanmax(sf.time_data)
sf.burn_in_time = jnp.arange(sf.start - 1000 - 1, sf.start - 1)
sf.oversample = 1008
sf.time_data_fine = jnp.linspace(sf.start - 1, sf.end + 1, int(sf.oversample * (sf.end - sf.start + 2)))
sf.burnin_oversample = 1
sf.offset = 0
sf.annual = jnp.arange(sf.start, sf.end + 1)
sf.mask = jnp.in1d(sf.annual, sf.time_data)
sf.growth = jnp.array([0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0])

INFO[2022-02-01 11:19:01,301]: Unable to initialize backend 'tpu_driver': Not found: Unable to find driver in registry given worker: 
INFO[2022-02-01 11:19:01,303]: Unable to initialize backend 'gpu': Not found: Could not find registered platform with name: "cuda". Available platform names are: Host Interpreter
INFO[2022-02-01 11:19:01,304]: Unable to initialize backend 'tpu': Invalid argument: TpuPlatform is not available.
WARNING[2022-02-01 11:19:01,305]: No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [3]:
mf = fitting.MultiFitter()
sf.compile_production_model(model="simple_sinusoid")
mf.add_SingleFitter(sf)
mf.add_SingleFitter(sf)
mf.compile()

In [4]:
out = mf.multi_likelihood(params=jnp.array([205., 1. / 12, jnp.pi / 2., 81. / 12]))
out

DeviceArray(-271748.72808792, dtype=float64)

In [5]:
out = sf.simple_sinusoid(200, jnp.array([205., 1./12, jnp.pi/2., 81./12]))
out

DeviceArray(2.1823329, dtype=float64)

In [6]:
out = sf.flexible_sinusoid(200, jnp.array([205., 1./12, jnp.pi/2., 81./12, 0.1]))
out

DeviceArray(2.04813439, dtype=float64)

In [7]:
sf.compile_production_model(model="simple_sinusoid")
a = sf.dc14(params=jnp.array([205., 1. / 12, jnp.pi / 2., 81./12]))
sf.compile_production_model(model="flexible_sinusoid")
b = sf.dc14(params=(jnp.array([205., 1./12, jnp.pi/2., 81./12, 0.1])))
sf.compile_production_model(model="control_points")
c = sf.dc14(params=jnp.ones(sf.control_points_time.size))
a, b, c

(DeviceArray([ 0.98204272,  1.52883272,  1.58545865,  1.13393995,
               0.3176567 , 12.99251179, 15.10262893, 15.1004269 ,
              14.58562105, 14.17481503], dtype=float64),
 DeviceArray([ 0.54557929,  0.84935151,  0.88081036,  0.62996664,
               0.17647595, 13.26103646, 15.69767009, 15.83511032,
              15.22872888, 14.52420871], dtype=float64),
 DeviceArray([-126.61096304, -126.64859288, -126.69988996, -126.75072931,
              -126.80273836, -126.85424935, -126.90696527, -126.95362958,
              -127.03376206, -127.62190907], dtype=float64))

dc14 fine

In [8]:
sf.compile_production_model(model="simple_sinusoid")
a = sf.dc14_fine(params=jnp.array([205., 1. / 12, jnp.pi / 2., 81./12]))[-9:]
sf.compile_production_model(model="flexible_sinusoid")
b = sf.dc14_fine(params=jnp.array([205., 1./12, jnp.pi/2., 81./12, 0.1]))[-9:]
sf.compile_production_model(model="control_points")
c = sf.dc14_fine(params=jnp.ones(sf.control_points_time.size))[-9:]
a, b, c

(DeviceArray([13.88683504, 13.88667417, 13.88651321, 13.88635218,
              13.88619107, 13.88602988, 13.88586861, 13.88570727,
              13.88554584], dtype=float64),
 DeviceArray([13.63361525, 13.63306659, 13.63251794, 13.6319693 ,
              13.63142066, 13.63087202, 13.63032339, 13.62977477,
              13.62922615], dtype=float64),
 DeviceArray([-129.75148614, -129.75330728, -129.75512883, -129.7569508 ,
              -129.75877317, -129.76059596, -129.76241915, -129.76424276,
              -129.76606678], dtype=float64))

In [9]:
sf.compile_production_model(model="simple_sinusoid")
out = sf.log_likelihood(jnp.array([205., 1. / 12, jnp.pi / 2., 81./12]))
out

DeviceArray(-135889.35596721, dtype=float64)

In [10]:
sf.compile_production_model(model="control_points")
out = sf.log_likelihood_gp(jnp.ones(sf.control_points_time.size))
out

DeviceArray(-10.21255349, dtype=float64)

In [12]:
sf.compile_production_model(model="control_points")
out = sf.log_joint_likelihood_gp(jnp.ones(sf.control_points_time.size), jnp.zeros((sf.control_points_time.size)),
                                                        jnp.ones(sf.control_points_time.size) * 100)
out

DeviceArray(-4818.88864661, dtype=float64)

In [13]:
out = sf.grad_neg_log_joint_likelihood_gp(jnp.ones(sf.control_points_time.size))
out

DeviceArray([40807.33529931,   601.15067189,   453.70227677,
               399.07149295,   334.40422425,   279.84279019,
               232.99899038,   147.72477284,   161.56303745],            dtype=float64)